# Temporary Worksapce

To test things out

In [1]:
using Muon
using RData
using RCall
using Random
using DataFrames
using Statistics
using Combinatorics

In [2]:
mdata = readh5mu("../data/mudata.h5mu")
adata = readh5ad("../data/SCT.h5ad")
spatial = readh5ad("../data/Spatial.h5ad")
lr_network = load("../data/lr_network.rds");

# display(first(adata.obs, 5))
# display(first(adata.var, 5))
adata

AnnData object 2185 ✕ 18910

In [3]:
# Create LR_Pairs column
lr_network[!, :LR_Pairs] = string.(lr_network.from, "_", lr_network.to);
lr_network = lr_network[:, [:from, :to, :LR_Pairs]]

# Filter lr_network based on conditions
from_filter = in.(lr_network[:, :from], Ref(adata.var_names))
to_filter = in.(lr_network[:, :to], Ref(adata.var_names))
all_LR_network = lr_network[from_filter .& to_filter, :]

# To reduce the computation time for this example, we randomly sample from the whole dataset of LR interactions

# all_LR_network = all_LR_network[shuffle(1:size(all_LR_network_exp, 1)), :]
all_LR_network = all_LR_network[1:min(3000, end), :]

# Extract unique genes and common genes
all_LR_genes = unique(vcat(all_LR_network[:, :from], all_LR_network[:, :to]))
all_LR_genes_comm = intersect(all_LR_genes, collect(adata.var_names));

# Create LR.pairs and LR.pairs.AllCombos
LR_pairs = all_LR_network[:, :LR_Pairs]
all_combos = [join(combo, "_") for combo in combinations(all_LR_genes_comm, 2)];

In [4]:
# r_lr_pairs = ["CSF1_CSF2RA", "WNT11_FZD3", "WNT11_FZD6", "WNT11_MUSK", "PTN_ALK", "WNT3_FZD3", "IHH_PTCH2", "SHH_PTCH2", "CCL26_CCR5", "DHH_PTCH1", "ADAM17_IL6R", "CCL26_CX3CR1", "MDK_NOTCH2", "IL10_IFNAR2", "PENK_OPRL1", "CNTF_CRLF1", "EDN1_EDNRB", "TGFB1_ITGB6", "VTN_ITGB6", "TGFB1_ERBB2", "TGFA_ERBB2", "ANGPT2_ITGA5", "COL1A1_ITGA5", "APP_SDC1", "APP_LRP1", "APP_KDR", "APP_NCSTN", "APP_NGFR", "APP_GPC1", "KNG1_RXFP4", "SPP1_SDC1", "CTSG_SDC1", "CALR_LRP1", "THBS1_LRP1", "CTSG_LRP1", "MDK_LRP1", "APOE_LRP1", "TGFB1_SDC2", "HGF_SDC2", "PF4_SDC2", "SERPINA1_SDC2", "TNFSF13_SDC2", "HSP90B1_TNFRSF1B", "CALM1_TNFRSF1B", "GZMB_NOTCH1", "CSF2_IL3RA", "MDK_SDC4", "TGFB3_SDC4", "PSEN1_NOTCH4", "PSEN1_NOTCH3", "PSEN1_NOTCH2", "FN1_TSHR", "FN1_ITGB5", "MMP9_CD44", "MMP9_EPHB2", "TNFSF9_TRAF2", "TNFSF4_TRAF2", "CALM1_TRAF2", "TNF_PTPRZ1", "CCL22_CCR7", "GDF9_ACVR2A", "FGF7_FGFR3", "FGF7_FGFR4", "WNT5A_FZD1", "COL1A1_CD36", "CALM1_FAS", "TNFSF13_FAS", "NTN1_NEO1", "CCL3_ACKR4", "CTSG_CXCR4", "TGFB1_BMPR1A", "GDF9_BMPR1A", "VTN_PLAUR", "KNG1_PLAUR", "CCL2_CCR10", "TDGF1_ACVR1C", "CCL13_ACKR4", "CCL11_ACKR4", "CCL8_ACKR4", "CXCL13_ACKR4", "CCL5_ACKR4", "CCL4_ACKR4", "EFNA1_EPHB1", "TGFB1_ITGAV", "TGFB1_ITGB8", "PROS1_RPSA", "TNFSF13_TNFRSF1A", "CALM1_TNFRSF1A", "CCL5_CXCR5", "CCL4_CXCR5", "PTN_PTPRS", "CTSG_F2RL1", "GDF5_ACVR1", "GSTP1_IL7R", "GDF5_ACVR2B", "FGF2_CD44", "VEGFA_PTPRB", "CCL25_CCR10", "TF_IL7R", "A2M_TNFRSF14", "CXCL13_CCR10", "PF4_CXCR3", "CXCL13_CXCR3", "NPY_MC4R", "PLAU_IGF2R", "BDNF_SORT1", "LRPAP1_SORT1", "NTS_SORT1", "SAA1_FPR1", "LRPAP1_SORL1", "INHBA_TGFBR3", "CALM1_ADCYAP1R1", "CALM1_GP6", "CALM1_GLP1R", "CALM1_VIPR1", "APOE_SCARB1", "VEGFA_EPHB2", "VEGFA_PTPRZ1", "VEGFA_GPC1", "C3_C5AR2", "LRPAP1_LRP8", "GCG_GIPR", "LRPAP1_VLDLR", "KNG1_GP1BA", "TNFSF13_TNFRSF14", "NPPA_NPR2", "EDN3_EDNRA", "CALM1_PTH1R", "PROC_THBD", "LRRC4_NTNG1", "INHBB_ACVR1C", "CORT_CXCR6", "CORT_GALR2", "CORT_FPR1", "CORT_C5AR1", "CORT_CXCR1", "CORT_CXCR2", "CORT_FPR2", "CORT_ACKR3", "CORT_NPY1R", "CORT_HTR1D", "CORT_BDKRB2", "CORT_CCR1", "CORT_CCR7", "CORT_CXCR5", "CORT_APLNR", "CORT_OPRD1", "CORT_OPRL1", "CORT_CCR2", "CORT_CCR10", "CORT_BDKRB1", "CORT_GALR1", "CORT_NPY2R", "CORT_CXCR3", "CORT_CCR5", "CORT_CCR6", "CORT_CCR9", "CORT_CXCR4", "CORT_P2RY14", "CORT_C3AR1", "CORT_RXFP4", "CORT_LPAR1", "CORT_NMUR1", "CORT_LPAR2", "CORT_PTGDR2", "LEFTY2_ACVR2A", "LEFTY2_ACVR1B", "LEFTY2_ACVR1C", "AGRN_ITGAV", "AGRN_LRP1", "AGRN_PTPRS", "AGRN_LRP8", "CCL21_GALR2", "CCL21_FPR1", "CCL21_C5AR1", "CCL21_CXCR1", "CCL21_CXCR2", "CCL21_FPR2", "CCL21_ACKR3", "CCL21_NPY1R", "CCL21_HTR1D", "CCL21_BDKRB2", "CCL21_SSTR1", "CCL21_SSTR2", "CCL21_CCR1", "CCL21_CXCR5", "CCL21_SSTR3", "CCL21_APLNR", "CCL21_OPRD1", "CCL21_OPRL1", "CCL21_CCR2", "CCL21_CCR10", "CCL21_BDKRB1", "CCL21_GALR1", "CCL21_NPY2R", "CCL21_CCR5", "CCL21_CCR6", "CCL21_CCR9", "CCL21_CXCR4", "CCL21_P2RY14", "CCL21_C3AR1", "CCL21_RXFP4", "CCL21_LPAR1", "CCL21_NMUR1", "CCL21_LPAR2", "CCL21_PTGDR2", "NRG2_EGFR", "CXCL11_GALR2", "CXCL11_FPR1", "CXCL11_C5AR1", "CXCL11_CXCR1", "CXCL11_CXCR2", "CXCL11_FPR2", "CXCL11_NPY1R", "CXCL11_HTR1D", "CXCL11_BDKRB2", "CXCL11_SSTR1", "CXCL11_SSTR2", "CXCL11_CCR1", "CXCL11_CCR7", "CXCL11_CXCR5", "CXCL11_SSTR3", "CXCL11_APLNR", "CXCL11_OPRD1", "CXCL11_OPRL1", "CXCL11_CCR2", "CXCL11_CCR10", "CXCL11_BDKRB1", "CXCL11_GALR1", "CXCL11_NPY2R", "CXCL11_CCR5", "CXCL11_CCR6", "CXCL11_CCR9", "CXCL11_CXCR4", "CXCL11_P2RY14", "CXCL11_C3AR1", "CXCL11_RXFP4", "CXCL11_LPAR1", "CXCL11_NMUR1", "CXCL11_LPAR2", "CXCL11_PTGDR2", "TNFSF11_TRAF2", "NPFF_TACR2", "NPFF_EDNRB", "NPFF_FPR2", "NPFF_EDNRA", "NPFF_TACR1", "NPFF_F2R", "NPFF_HTR2A", "NPFF_BDKRB2", "NPFF_AGTR1", "NPFF_OXTR", "NPFF_NTSR1", "NPFF_HRH1", "NPFF_AVPR1A", "NPFF_HTR2B", "NPFF_XCR1", "NPFF_BDKRB1", "NPFF_GCGR", "NPFF_AVPR1B", "NPFF_F2RL1", "NPFF_LPAR1", "NPFF_KISS1R", "NPFF_F2RL3", "NPFF_NMUR1", "NPFF_LPAR2", "CCL25_GALR2", "CCL25_FPR1", "CCL25_C5AR1", "CCL25_CXCR1", "CCL25_CXCR2", "CCL25_FPR2", "CCL25_ACKR3", "CCL25_NPY1R", "CCL25_HTR1D", "CCL25_BDKRB2", "CCL25_SSTR1", "CCL25_SSTR2", "CCL25_CCR1", "CCL25_CCR7", "CCL25_CXCR5", "CCL25_SSTR3", "CCL25_APLNR", "CCL25_OPRD1", "CCL25_OPRL1", "CCL25_CCR2", "CCL25_BDKRB1", "CCL25_GALR1", "CCL25_NPY2R", "CCL25_CXCR3", "CCL25_CCR5", "CCL25_CCR6", "CCL25_CXCR4", "CCL25_P2RY14", "CCL25_C3AR1", "CCL25_RXFP4", "CCL25_LPAR1", "CCL25_NMUR1", "CCL25_LPAR2", "CCL25_PTGDR2", "DHH_SMO", "TNFSF12_TRAF2", "HCRT_TACR2", "HCRT_EDNRB", "HCRT_FPR2", "HCRT_EDNRA", "HCRT_TACR1", "HCRT_F2R", "HCRT_HTR2A", "HCRT_BDKRB2", "HCRT_AGTR1", "HCRT_OXTR", "HCRT_NTSR1", "HCRT_HRH1", "HCRT_AVPR1A", "HCRT_HTR2B", "HCRT_XCR1", "HCRT_BDKRB1", "HCRT_GCGR", "HCRT_AVPR1B", "HCRT_F2RL1", "HCRT_LPAR1", "HCRT_KISS1R", "HCRT_F2RL3", "HCRT_NPFFR1", "HCRT_NMUR1", "HCRT_LPAR2", "EFNA2_EPHB2", "EFNA2_TGFBR1", "EFNA2_EPHB3", "EFNA2_EPHB4", "EFNA2_EPHB1", "CXCL13_FPR1", "CXCL13_C5AR1", "CXCL13_CXCR1", "CXCL13_CXCR2", "CXCL13_FPR2", "CXCL13_ACKR3", "CXCL13_NPY1R", "CXCL13_HTR1D", "CXCL13_BDKRB2", "CXCL13_SSTR1", "CXCL13_SSTR2", "CXCL13_CCR1", "CXCL13_CCR7", "CXCL13_SSTR3", "CXCL13_APLNR", "CXCL13_OPRD1", "CXCL13_OPRL1", "CXCL13_CCR2", "CXCL13_BDKRB1", "CXCL13_GALR1", "CXCL13_NPY2R", "CXCL13_CCR5", "CXCL13_CCR6", "CXCL13_CCR9", "CXCL13_CXCR4", "CXCL13_P2RY14", "CXCL13_C3AR1", "CXCL13_RXFP4", "CXCL13_LPAR1", "CXCL13_NMUR1", "CXCL13_LPAR2", "CXCL13_PTGDR2", "PSPN_GFRA3", "PSPN_GFRA1", "SLIT1_DCC", "SEMA7A_ITGA1", "SLIT2_DCC", "SLIT2_PTCH1", "SLIT2_ROBO3", "GDF11_TGFBR1", "GDF11_ACVR1C", "NTN1_ITGB4", "NTN1_ITGA6", "NTN1_ACVR1B", "NTN1_TGFBR1", "NTN1_NOTCH1", "NTN1_NOTCH2", "NTN1_PTCH1", "NTN1_UNC5A", "NTN1_ACVR1C", "NTN1_ROBO2", "NTN1_NOTCH3", "NTN1_PTCH2", "WNT11_FZD1", "PLAU_CD44", "AGT_TACR2", "AGT_FPR1", "AGT_C5AR1", "AGT_EDNRB", "AGT_CXCR1", "AGT_CXCR2", "AGT_FPR2", "AGT_EDNRA", "AGT_TACR1", "AGT_ACKR3", "AGT_F2R", "AGT_NPY1R", "AGT_HTR1D", "AGT_HTR2A", "AGT_BDKRB2", "AGT_OXTR", "AGT_SSTR1", "AGT_SSTR2", "AGT_NTSR1", "AGT_CCR1", "AGT_CCR7", "AGT_CXCR5", "AGT_SSTR3", "AGT_HRH1", "AGT_APLNR", "AGT_AVPR1A", "AGT_OPRD1", "AGT_OPRL1", "AGT_HTR2B", "AGT_CCR2", "AGT_CCR10", "AGT_XCR1", "AGT_BDKRB1", "AGT_GALR1", "AGT_GCGR", "AGT_AVPR1B", "AGT_NPY2R", "AGT_CXCR3", "AGT_CCR5", "AGT_CCR6", "AGT_CCR9", "AGT_F2RL1", "AGT_CXCR4", "AGT_P2RY14", "AGT_C3AR1", "AGT_RXFP4", "AGT_LPAR1", "AGT_KISS1R", "AGT_F2RL3", "AGT_NPFFR1", "AGT_NMUR1", "AGT_LPAR2", "AGT_PTGDR2", "A2M_TYRO3", "C3_FPR1", "C3_C5AR1", "C3_CXCR1", "C3_CXCR2", "C3_FPR2", "C3_ACKR3", "C3_NPY1R", "C3_HTR1D", "C3_BDKRB2", "C3_SSTR1", "C3_SSTR2", "C3_CCR1", "C3_CCR7", "C3_CXCR5", "C3_SSTR3", "C3_APLNR", "C3_OPRD1", "C3_OPRL1", "C3_CCR2", "C3_CCR10", "C3_BDKRB1", "C3_GALR1", "C3_NPY2R", "C3_CXCR3", "C3_CCR5", "C3_CCR6", "C3_CCR9", "C3_CXCR4", "C3_P2RY14", "C3_RXFP4", "C3_LPAR1", "C3_NMUR1", "C3_LPAR2", "C3_PTGDR2", "C5_FPR1", "C5_CXCR1", "C5_CXCR2", "C5_FPR2", "C5_ACKR3", "C5_NPY1R", "C5_HTR1D", "C5_BDKRB2", "C5_SSTR1", "C5_SSTR2", "C5_CCR1", "C5_CCR7", "C5_CXCR5", "C5_SSTR3", "C5_APLNR", "C5_OPRD1", "C5_OPRL1", "C5_CCR2", "C5_CCR10", "C5_BDKRB1", "C5_GALR1", "C5_NPY2R", "C5_CXCR3", "C5_CCR5", "C5_CCR6"]
# setdiff(r_lr_pairs, LR_pairs)
# setdiff(LR_pairs, r_lr_pairs)

In [5]:
spatial_object = mdata["Spatial"]
COI = ["CC4"]
Condition = unique(adata.obs[!, "orig.ident"])
LR_list = all_LR_genes_comm
LR_pairs = LR_pairs
exp_threshold = 1
corr_threshold = 0.2

0.2

In [6]:
# Extract the expression matrix from spatial_object
spatial_object_exp = spatial_object.layers["counts"]
spatial_object_exp_norm = spatial_object.X
spatial_object_exp_norm

2185×36601 SparseArrays.SparseMatrixCSC{Float64, Int32} with 5321609 stored entries:
⎡⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎤
⎣⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎦

In [7]:
# Subset the expression matrix for the interested ligands and receptors
# spatial_obj_exp_LR_subset_raw = spatial_object_exp[:, indexin(LR_list, spatial_object.var.name)]
spatial_obj_exp_LR_subset_raw = spatial_object[:, in.(spatial_object.var.name, Ref(LR_list))]
# spatial_obj_exp_LR_subset_raw.layers["counts"]

AnnData object 2185 ✕ 758

In [8]:
# Binarize the expression matrix based on the expression threshold
spatial_obj_exp_LR_subset_raw_binary = spatial_obj_exp_LR_subset_raw.layers["counts"] .> exp_threshold
spatial_obj_exp_LR_subset_raw.layers["binary"] = spatial_obj_exp_LR_subset_raw_binary

LR_subset_raw_binary_mask_col = vec(sum(spatial_obj_exp_LR_subset_raw_binary, dims=1) .> 0)
LR_subset_raw_binary_mask_row = vec(sum(spatial_obj_exp_LR_subset_raw_binary, dims=2) .> 0)

LR_presence_absence = spatial_obj_exp_LR_subset_raw[LR_subset_raw_binary_mask_row, LR_subset_raw_binary_mask_col]
# LR_presence_absence.layers["binary"] = spatial_obj_exp_LR_subset_raw_binary[LR_subset_raw_binary_mask_row, LR_subset_raw_binary_mask_col]
LR_presence_absence_mat = LR_presence_absence.layers["binary"]

2148×567 SparseArrays.SparseMatrixCSC{Bool, Int32} with 86492 stored entries:
⎡⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⎤
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⎥
⎣⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⎦

In [9]:
# # Binarize the expression matrix based on the expression threshold
# spatial_obj_exp_LR_subset_raw_binary = spatial_obj_exp_LR_subset_raw.layers["counts"] .> exp_threshold
# LR_subset_raw_binary_mask_col = vec(sum(spatial_obj_exp_LR_subset_raw_binary, dims=1) .> 0)
# LR_subset_raw_binary_mask_row = vec(sum(spatial_obj_exp_LR_subset_raw_binary, dims=2) .> 0)
# spatial_obj_exp_LR_subset_raw_binary = spatial_obj_exp_LR_subset_raw_binary[:, LR_subset_raw_binary_mask_col]
# spatial_obj_exp_LR_subset_raw_binary = spatial_obj_exp_LR_subset_raw_binary[LR_subset_raw_binary_mask_row, :]
# LR_presence_absence_mat = spatial_obj_exp_LR_subset_raw_binary
# LR_presence_absence_mat

In [10]:
# Create a mask to filter rows based on COI and Condition
mask = (spatial_object.obs[:, "CompositionCluster_CC"] .∈ Ref(COI)) .& (spatial_object.obs[:, "orig.ident"] .∈ Ref(Condition))
COI_spots = spatial_object.obs_names[mask]
rest_of_spots = setdiff(spatial_object.obs_names, COI_spots)
display(length(COI_spots))
display(length(rest_of_spots))

14

2171

In [11]:
println("Calculating L-R pairs correlation")
COI_cors_adata = spatial_obj_exp_LR_subset_raw[mask, :]
COI_cors = cor(Array(COI_cors_adata.layers["counts"]))
COI_cors[isnan.(COI_cors)] .= 0.0
COI_cors
size(COI_cors)

Calculating L-R pairs correlation

(758, 758)

In [12]:
COI_cors_adata.var.name

758-element Vector{String}:
 "AGRN"
 "TNFRSF4"
 "TNFRSF14"
 "TNFRSF25"
 "TNFRSF9"
 "CORT"
 "NPPA"
 "TNFRSF8"
 "TNFRSF1B"
 "EPHA2"
 ⋮
 "APLN"
 "CD40LG"
 "FGF13"
 "PLXNB3"
 "L1CAM"
 "AVPR2"
 "PLXNA3"
 "F8"
 "NLGN4Y"

In [13]:
# r_rownames = ["CSF1", "PTN", "IHH", "SHH", "ADAM17", "MDK", "IL10", "EDN1", "TGFB1", "TGFA", "ANGPT2", "COL1A1", "APP", "SPP1", "CTSG", "CALR", "THBS1", "APOE", "HGF", "PF4", "SERPINA1", "TNFSF13", "HSP90B1", "CALM1", "GZMB", "TGFB3", "PSEN1", "FN1", "MMP9", "TNFSF4", "CCL22", "FGF7", "WNT5A", "NTN1", "CCL3", "CCL2", "CCL13", "CCL11", "CCL8", "CXCL13", "CCL5", "CCL4", "EFNA1", "PROS1", "GSTP1", "FGF2", "VEGFA", "TF", "A2M", "PLAU", "LRPAP1", "NTS", "SAA1", "INHBA", "C3", "GCG", "EDN3", "PROC", "INHBB", "AGRN", "CCL21", "CXCL11", "NPFF", "TNFSF12", "HCRT", "EFNA2", "PSPN", "SEMA7A", "GDF11", "AGT", "C5", "CSF2RA", "FZD3", "FZD6", "MUSK", "PTCH2", "CCR5", "PTCH1", "IL6R", "NOTCH2", "IFNAR2", "CRLF1", "EDNRB", "ITGB6", "ERBB2", "ITGA5", "SDC1", "LRP1", "KDR", "NCSTN", "NGFR", "GPC1", "SDC2", "TNFRSF1B", "NOTCH1", "IL3RA", "SDC4", "NOTCH4", "NOTCH3", "ITGB5", "CD44", "EPHB2", "TRAF2", "CCR7", "ACVR2A", "FGFR3", "FGFR4", "FZD1", "CD36", "FAS", "NEO1", "ACKR4", "CXCR4", "BMPR1A", "PLAUR", "CCR10", "ACVR1C", "ITGAV", "ITGB8", "RPSA", "TNFRSF1A", "F2RL1", "ACVR1", "IL7R", "PTPRB", "TNFRSF14", "CXCR3", "MC4R", "IGF2R", "SORT1", "FPR1", "SORL1", "TGFBR3", "VIPR1", "SCARB1", "C5AR2", "LRP8", "GIPR", "VLDLR", "NPR2", "EDNRA", "PTH1R", "THBD", "CXCR6", "C5AR1", "ACKR3", "NPY1R", "BDKRB2", "CCR1", "APLNR", "CCR2", "BDKRB1", "CCR6", "P2RY14", "C3AR1", "LPAR1", "LPAR2", "ACVR1B", "SSTR1", "SSTR2", "EGFR", "TACR2", "TACR1", "F2R", "HRH1", "AVPR1A", "GCGR", "F2RL3", "TGFBR1", "EPHB3", "EPHB4", "GFRA3", "GFRA1", "ITGA1", "ROBO3", "ITGB4", "ITGA6"]
# setdiff(r_rownames, coocur_COI_exp_row_names)

In [14]:
# temp = DataFrame(hcat(coocur_COI_exp_row_names, sum.(eachrow(coocur_COI_exp))), :auto)
# temp[temp[:, "x1"] .∈ Ref(["CSF1", "SAA1", "IL7R", "GSTP1"]), :]

In [15]:
println("Preparing for cooccurrence")
common_spots = intersect(LR_presence_absence.obs_names, COI_spots)
coocur_COI = LR_presence_absence[common_spots, :]
coocur_COI_exp = DataFrame(Matrix(transpose(coocur_COI.layers["binary"])), common_spots)
coocur_COI_exp_row_names = coocur_COI.var.name
# Values are same as in R code, but the positions of rows are different

Preparing for cooccurrence

567-element Vector{String}:
 "AGRN"
 "TNFRSF4"
 "TNFRSF14"
 "TNFRSF25"
 "TNFRSF9"
 "TNFRSF1B"
 "EPHA2"
 "HSPG2"
 "WNT4"
 "EPHB2"
 ⋮
 "CSF2RA"
 "IL3RA"
 "CD99"
 "ATP6AP2"
 "EFNB1"
 "IL2RG"
 "CXCR3"
 "L1CAM"
 "PLXNA3"

In [16]:
@rput coocur_COI_exp coocur_COI_exp_row_names
println("Cooccurrence calculation starts...")
R"""
ISCHIA.cooccur <- function (mat, type = "spp_site", thresh = TRUE, spp_names = FALSE,
          true_rand_classifier = 0.1, prob = "hyper", site_mask = NULL,
          only_effects = FALSE, eff_standard = TRUE, eff_matrix = FALSE)
{
  if (type == "spp_site") {
    spp_site_mat <- mat
  }
  if (type == "site_spp") {
    spp_site_mat <- t(mat)
  }
  if (spp_names == TRUE) {
    spp_key <- data.frame(num = 1:nrow(spp_site_mat), spp = row.names(spp_site_mat))
  }
  if (!is.null(site_mask)) {
    if (nrow(site_mask) == nrow(spp_site_mat) & ncol(site_mask) ==
        ncol(spp_site_mat)) {
      N_matrix <- create.N.matrix(site_mask)
    }
    else {
      stop("Incorrect dimensions for site_mask, aborting.")
    }
  }
  else {
    site_mask <- matrix(data = 1, nrow = nrow(spp_site_mat),
                        ncol = ncol(spp_site_mat))
    N_matrix <- matrix(data = ncol(spp_site_mat), nrow = nrow(spp_site_mat),
                       ncol = nrow(spp_site_mat))
  }
  spp_site_mat[spp_site_mat > 0] <- 1
  tsites <- ncol(spp_site_mat)
  nspp <- nrow(spp_site_mat)
  spp_pairs <- choose(nspp, 2)
  incidence <- prob_occur <- obs_cooccur <- prob_cooccur <- exp_cooccur <- matrix(nrow = spp_pairs,
                                                                                  ncol = 3)
  incidence <- prob_occur <- matrix(nrow = nrow(N_matrix),
                                    ncol = ncol(N_matrix))

  for (spp in 1:nspp) {
    if (spp < nspp) {
      for (spp_next in (spp + 1):nspp) {
        incidence[spp, spp_next] <- sum(site_mask[spp,
        ] * site_mask[spp_next, ] * mat[spp, ])
        incidence[spp_next, spp] <- sum(site_mask[spp,
        ] * site_mask[spp_next, ] * mat[spp_next, ])
      }
    }
  }
  prob_occur <- incidence/N_matrix
  pb <- txtProgressBar(min = 0, max = (nspp + nrow(obs_cooccur)),
                       style = 3)
  row <- 0
  for (spp in 1:nspp) {
    if (spp < nspp) {
      for (spp_next in (spp + 1):nspp) {
        pairs <- sum(as.numeric(mat[spp, site_mask[spp,
        ] * site_mask[spp_next, ] == 1] == 1 & mat[spp_next,
                                                   site_mask[spp, ] * site_mask[spp_next, ] ==
                                                     1] == 1))
        row <- row + 1
        obs_cooccur[row, 1] <- spp
        obs_cooccur[row, 2] <- spp_next
        obs_cooccur[row, 3] <- pairs
        prob_cooccur[row, 1] <- spp
        prob_cooccur[row, 2] <- spp_next
        prob_cooccur[row, 3] <- prob_occur[spp, spp_next] *
          prob_occur[spp_next, spp]
        exp_cooccur[row, 1] <- spp
        exp_cooccur[row, 2] <- spp_next
        exp_cooccur[row, 3] <- prob_cooccur[row, 3] *
          N_matrix[spp, spp_next]
      }
    }
    setTxtProgressBar(pb, spp)
  }
  if (thresh == TRUE) {
    n_pairs <- nrow(prob_cooccur)
    prob_cooccur <- prob_cooccur[exp_cooccur[, 3] >= 1, ]
    obs_cooccur <- obs_cooccur[exp_cooccur[, 3] >= 1, ]
    exp_cooccur <- exp_cooccur[exp_cooccur[, 3] >= 1, ]
    n_omitted <- n_pairs - nrow(prob_cooccur)
    pb <- txtProgressBar(min = 0, max = (nspp + nrow(obs_cooccur)),
                         style = 3)
  }
  output <- data.frame(matrix(nrow = 0, ncol = 9))
  colnames(output) <- c("sp1", "sp2", "sp1_inc", "sp2_inc",
                        "obs_cooccur", "prob_cooccur", "exp_cooccur", "p_lt",
                        "p_gt")
  for (row in 1:nrow(obs_cooccur)) {
    sp1 <- obs_cooccur[row, 1]
    sp2 <- obs_cooccur[row, 2]
    sp1_inc <- incidence[sp1, sp2]
    sp2_inc <- incidence[sp2, sp1]
    max_inc <- max(sp1_inc, sp2_inc)
    min_inc <- min(sp1_inc, sp2_inc)
    nsite <- N_matrix[sp1, sp2]
    psite <- as.numeric(nsite + 1)
    prob_share_site <- rep(x = 0, times = psite)
    if (prob == "hyper") {
      if (only_effects == FALSE) {
        all.probs <- phyper(0:min_inc, min_inc, nsite -
                              min_inc, max_inc)
        prob_share_site[1] <- all.probs[1]
        for (j in 2:length(all.probs)) {
          prob_share_site[j] <- all.probs[j] - all.probs[j -
                                                           1]
        }
      }
      else {
        for (j in 0:nsite) {
          if ((sp1_inc + sp2_inc) <= (nsite + j)) {
            if (j <= min_inc) {
              prob_share_site[(j + 1)] <- 1
            }
          }
        }
      }
    }
    if (prob == "comb") {
      if (only_effects == FALSE) {
        for (j in 0:nsite) {
          if ((sp1_inc + sp2_inc) <= (nsite + j)) {
            if (j <= min_inc) {
              prob_share_site[(j + 1)] <- coprob(max_inc = max_inc,
                                                 j = j, min_inc = min_inc, nsite = nsite)
            }
          }
        }
      }
      else {
        for (j in 0:nsite) {
          if ((sp1_inc + sp2_inc) <= (nsite + j)) {
            if (j <= min_inc) {
              prob_share_site[(j + 1)] <- 1
            }
          }
        }
      }
    }
    p_lt <- 0
    p_gt <- 0
    for (j in 0:nsite) {
      if (j <= obs_cooccur[row, 3]) {
        p_lt <- prob_share_site[(j + 1)] + p_lt
      }
      if (j >= obs_cooccur[row, 3]) {
        p_gt <- prob_share_site[(j + 1)] + p_gt
      }
      if (j == obs_cooccur[row, 3]) {
        p_exactly_obs <- prob_share_site[(j + 1)]
      }
    }
    p_lt <- round(p_lt, 5)
    p_gt <- round(p_gt, 5)
    p_exactly_obs <- round(p_exactly_obs, 5)
    prob_cooccur[row, 3] <- round(prob_cooccur[row, 3], 3)
    exp_cooccur[row, 3] <- round(exp_cooccur[row, 3], 1)
    output[row, ] <- c(sp1, sp2, sp1_inc, sp2_inc, obs_cooccur[row,
                                                               3], prob_cooccur[row, 3], exp_cooccur[row, 3], p_lt,
                       p_gt)
    setTxtProgressBar(pb, nspp + row)
  }
  close(pb)
  if (spp_names == TRUE) {
    sp1_name <- merge(x = data.frame(order = 1:length(output$sp1),
                                     sp1 = output$sp1), y = spp_key, by.x = "sp1", by.y = "num",
                      all.x = T, sort = FALSE)
    sp2_name <- merge(x = data.frame(order = 1:length(output$sp2),
                                     sp2 = output$sp2), y = spp_key, by.x = "sp2", by.y = "num",
                      all.x = T, sort = FALSE)
    output$sp1_name <- sp1_name[with(sp1_name, order(order)),
                                "spp"]
    output$sp2_name <- sp2_name[with(sp2_name, order(order)),
                                "spp"]
  }
  true_rand <- (nrow(output[(output$p_gt >= 0.05 & output$p_lt >=
                               0.05) & (abs(output$obs_cooccur - output$exp_cooccur) <=
                                          (tsites * true_rand_classifier)), ]))
  output_list <- list(call = match.call(), results = output,
                      positive = nrow(output[output$p_gt < 0.05, ]), negative = nrow(output[output$p_lt <
                                                                                              0.05, ]), co_occurrences = (nrow(output[output$p_gt <
                                                                                                                                        0.05 | output$p_lt < 0.05, ])), pairs = nrow(output),
                      random = true_rand, unclassifiable = nrow(output) - (true_rand +
                                                                             nrow(output[output$p_gt < 0.05, ]) + nrow(output[output$p_lt <
                                                                                                                                0.05, ])), sites = N_matrix, species = nspp, percent_sig = (((nrow(output[output$p_gt <
                                                                                                                                                                                                            0.05 | output$p_lt < 0.05, ])))/(nrow(output))) *
                        100, true_rand_classifier = true_rand_classifier)
  if (spp_names == TRUE) {
    output_list$spp_key <- spp_key
    output_list$spp.names = row.names(spp_site_mat)
  }
  else {
    output_list$spp.names = c(1:nrow(spp_site_mat))
  }
  if (thresh == TRUE) {
    output_list$omitted <- n_omitted
    output_list$pot_pairs <- n_pairs
  }
  class(output_list) <- "cooccur"
  if (only_effects == F) {
    output_list
  }
  else {
    effect.sizes(mod = output_list, standardized = eff_standard,
                 matrix = eff_matrix)
  }
}

row.names(coocur_COI_exp) <- coocur_COI_exp_row_names

cooccur_COI_res = ISCHIA.cooccur(mat=coocur_COI_exp, type="spp_site", thresh=TRUE, spp_names=TRUE)
"""
println("Cooccurrence calculation ended")

@rget cooccur_COI_res

Cooccurrence calculation starts...


  |                                                                      |   0%

  |======================================================================| 100%
Cooccurrence calculation ended


OrderedCollections.OrderedDict{Symbol, Any} with 16 entries:
  :call                 => :(var"ISCHIA.cooccur"($(Expr(:(=), :mat, :coocur_COI…
  :results              => 215×11 DataFrame…
  :positive             => 28
  :negative             => 0
  :co_occurrences       => 28
  :pairs                => 215
  :random               => 151
  :unclassifiable       => 36
  :sites                => [14 14 … 14 14; 14 14 … 14 14; … ; 14 14 … 14 14; 14…
  :species              => 567
  :percent_sig          => 13.0233
  :true_rand_classifier => 0.1
  :spp_key              => 567×2 DataFrame…
  :spp_names            => ["AGRN", "TNFRSF4", "TNFRSF14", "TNFRSF25", "TNFRSF9…
  :omitted              => 160246
  :pot_pairs            => 160461

In [17]:
println("Summary of cooccurrence results:")
display(R"summary(cooccur_COI_res)")

RObject{StrSxp}
                     Length Class      Mode     
call                      5 -none-     call     
results                  11 data.frame list     
positive                  1 -none-     numeric  
negative                  1 -none-     numeric  
co_occurrences            1 -none-     numeric  
pairs                     1 -none-     numeric  
random                    1 -none-     numeric  
unclassifiable            1 -none-     numeric  
sites                321489 -none-     numeric  
species                   1 -none-     numeric  
percent_sig               1 -none-     numeric  
true_rand_classifier      1 -none-     numeric  
spp_key                   2 data.frame list     
spp.names               567 -none-     character
omitted                   1 -none-     numeric  
pot_pairs                 1 -none-     numeric  


Summary of cooccurrence results:

In [18]:
println("Probability table of cooccurrence results:")
display(R"library(cooccur); prob.table(cooccur_COI_res)")

Probability table of cooccurrence results:

RObject{VecSxp}
    sp1 sp2 sp1_inc sp2_inc obs_cooccur prob_cooccur exp_cooccur    p_lt
1    68 118       3       5           3        0.077         1.1 1.00000
2    68 220       3       9           3        0.138         1.9 1.00000
3    68 221       3       6           2        0.092         1.3 0.94505
4    68 225       3       9           3        0.138         1.9 1.00000
5    68 230       3       7           2        0.107         1.5 0.90385
6    68 258       3       6           3        0.092         1.3 1.00000
7    68 297       3       5           2        0.077         1.1 0.97253
8    68 421       3       8           2        0.122         1.7 0.84615
9    68 499       3       5           3        0.077         1.1 1.00000
10   68 515       3       7           3        0.107         1.5 1.00000
11   85 118       4       5           3        0.102         1.4 0.99500
12   85 219       4       4           3        0.082         1.1 0.99900
13   85 220       4       9        

┌ Warning: RCall.jl: Warning in prob.table(cooccur_COI_res) :
│   The co-occurrence model was run using 'thresh = TRUE.' The probability table may not include all species pairs
└ @ RCall C:\Users\mraadam\.julia\packages\RCall\gOwEW\src\io.jl:172


In [19]:
# r_sp1 = ["CALR", "CALR", "CALR", "CALR", "THBS1", "THBS1", "THBS1", "THBS1", "THBS1", "THBS1", "THBS1", "THBS1", "THBS1", "THBS1", "THBS1", "HSP90B1", "HSP90B1", "HSP90B1", "HSP90B1", "HSP90B1", "GSTP1", "GSTP1", "GSTP1", "A2M", "A2M", "A2M", "A2M", "C3", "C3", "C3", "C3", "CCL21", "CCL21", "CCL21", "CXCR4", "RPSA"]
# r_sp2 = ["THBS1", "C3", "CCL21", "RPSA", "APOE", "HSP90B1", "CALM1", "GSTP1", "A2M", "SAA1", "C3", "CCL21", "CXCR4", "RPSA", "IL7R", "A2M", "C3", "CCL21", "CXCR4", "RPSA", "C3", "CCL21", "RPSA", "C3", "CCL21", "CXCR4", "RPSA", "CCL21", "CXCR4", "RPSA", "IL7R", "CXCR4", "RPSA", "IL7R", "RPSA", "IL7R"]

# j_sp1 = cooccur_COI_res[:results][!, :sp1_name]
# j_sp2 = cooccur_COI_res[:results][!, :sp2_name]

# display(setdiff(r_sp1, j_sp1))
# display(setdiff(r_sp1, j_sp2))
# display(setdiff(j_sp1, r_sp1))
# display(setdiff(j_sp1, r_sp2))
# display(setdiff(r_sp2, j_sp1))
# display(setdiff(r_sp2, j_sp2))
# display(setdiff(j_sp2, r_sp1))
# display(setdiff(j_sp2, r_sp2))

In [20]:
cooccur_res_df = cooccur_COI_res[:results]
# Add a 'pair' column to the result DataFrame
cooccur_res_df[!, :pair12] = string.(cooccur_res_df[!, :sp1_name], "_", cooccur_res_df[!, :sp2_name])
cooccur_res_df[!, :pair21] = string.(cooccur_res_df[!, :sp2_name], "_", cooccur_res_df[!, :sp1_name])
# rownames!(cooccur_COI_res, cooccur_COI_res[:pair]) doesn't work in Julia
cooccur_res_df

Row,sp1,sp2,sp1_inc,sp2_inc,obs_cooccur,prob_cooccur,exp_cooccur,p_lt,p_gt,sp1_name,sp2_name,pair12,pair21
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,String,String,String
1,68.0,118.0,3.0,5.0,3.0,0.077,1.1,1.0,0.02747,CALM2,RPSA,CALM2_RPSA,RPSA_CALM2
2,68.0,220.0,3.0,9.0,3.0,0.138,1.9,1.0,0.23077,CALM2,HLA-A,CALM2_HLA-A,HLA-A_CALM2
3,68.0,221.0,3.0,6.0,2.0,0.092,1.3,0.94505,0.38462,CALM2,HLA-E,CALM2_HLA-E,HLA-E_CALM2
4,68.0,225.0,3.0,9.0,3.0,0.138,1.9,1.0,0.23077,CALM2,HLA-DRA,CALM2_HLA-DRA,HLA-DRA_CALM2
5,68.0,230.0,3.0,7.0,2.0,0.107,1.5,0.90385,0.5,CALM2,CD24,CALM2_CD24,CD24_CALM2
6,68.0,258.0,3.0,6.0,3.0,0.092,1.3,1.0,0.05495,CALM2,NAMPT,CALM2_NAMPT,NAMPT_CALM2
7,68.0,297.0,3.0,5.0,2.0,0.077,1.1,0.97253,0.27473,CALM2,CCL21,CALM2_CCL21,CCL21_CALM2
8,68.0,421.0,3.0,8.0,2.0,0.122,1.7,0.84615,0.61538,CALM2,THBS1,CALM2_THBS1,THBS1_CALM2
9,68.0,499.0,3.0,5.0,3.0,0.077,1.1,1.0,0.02747,CALM2,C3,CALM2_C3,C3_CALM2


In [21]:
# R"plot(cooccur_COI_res)" 
# figure this out later, if needed 
# https://rdrr.io/cran/cooccur/src/R/plot.cooccur.R

In [22]:
all_cooccur_pairs = Set([cooccur_res_df[!, :pair12]; cooccur_res_df[!, :pair21]])
common_pairs = intersect(LR_pairs, all_cooccur_pairs)

2-element Vector{String}:
 "CCL21_CXCR4"
 "C3_CXCR4"

In [30]:
Set([cooccur_res_df.pair12; cooccur_res_df.pair21])

Set{String} with 430 elements:
  "RPS19_NAMPT"
  "HLA-A_RPS19"
  "IL7R_CD24"
  "IL7R_HLA-A"
  "IL7R_RPS19"
  "A2M_RPSA"
  "CALM2_C3"
  "NAMPT_IL7R"
  "NAMPT_HSP90B1"
  "RPS19_CALM2"
  "CALR_THBS1"
  "RARRES2_CD24"
  "HLA-A_HLA-E"
  "NAMPT_RPS19"
  "CCL21_RPS19"
  "HLA-A_IL10RA"
  "HLA-DRA_PSAP"
  "HSP90B1_PSAP"
  "CCL19_THBS1"
  ⋮ 

In [23]:
# r_output = ["CALR_THBS1", "CALR_C3", "CALR_CCL21", "CALR_RPSA", "THBS1_APOE", "THBS1_HSP90B1", "THBS1_CALM1", "THBS1_GSTP1", "THBS1_A2M", "THBS1_SAA1", "THBS1_C3", "THBS1_CCL21", "THBS1_CXCR4", "THBS1_RPSA", "THBS1_IL7R", "HSP90B1_A2M", "HSP90B1_C3", "HSP90B1_CCL21", "HSP90B1_CXCR4", "HSP90B1_RPSA", "GSTP1_C3", "GSTP1_CCL21", "GSTP1_RPSA", "A2M_C3", "A2M_CCL21", "A2M_CXCR4", "A2M_RPSA", "C3_CCL21", "C3_CXCR4", "C3_RPSA", "C3_IL7R", "CCL21_CXCR4", "CCL21_RPSA", "CCL21_IL7R", "CXCR4_RPSA", "RPSA_IL7R"]
# setdiff(r_output, Set([pair1; pair2]))
# setdiff(Set([pair1; pair2]), r_output)

In [24]:
# pair = "C3_CXCR4"
# pair_count = 0
# pair_count += 1

# println("$pair_count / $(length(common_pairs))")
# LR_pair_words = split(pair, "_")
# LR_pair_ligand = "Spatial:" * LR_pair_words[1]
# LR_pair_Receptor = "Spatial:" * LR_pair_words[2]

# ligand_exp_COI_mean = mean(spatial_object[COI_spots, LR_pair_ligand].X)
# ligand_exp_otherspots_mean = mean(spatial_object[rest_of_spots, LR_pair_ligand].X)
# ligand_FC = round(ligand_exp_COI_mean / ligand_exp_otherspots_mean, digits=4)

# Receptor_exp_COI_mean = mean(spatial_object[COI_spots, LR_pair_Receptor].X)
# Receptor_exp_otherspots_mean = mean(spatial_object[rest_of_spots, LR_pair_Receptor].X)
# Receptor_FC = round(Receptor_exp_COI_mean / Receptor_exp_otherspots_mean, digits=4)

# pair_p_gt = cooccur_res_df[(cooccur_res_df.pair12 .== pair) .| (cooccur_res_df.pair21 .== pair), :p_gt][1]
# ligand_index = findfirst(==(LR_pair_ligand), COI_cors_adata.var_names)
# receptor_index = findfirst(==(LR_pair_Receptor), COI_cors_adata.var_names)

# if pair_p_gt < 0.05 && COI_cors[ligand_index, receptor_index] > corr_threshold
#     added_row = DataFrame(from=[LR_pair_ligand], to=[LR_pair_Receptor], correlation=[COI_cors[ligand_index, receptor_index]], ligand_FC=[ligand_FC], Receptor_FC=[Receptor_FC])
#     append!(COI_enriched_LRs, added_row)
# end

COI_enriched_LRs = DataFrame(from=String[], to=String[], correlation=Float64[], ligand_FC=Float64[], Receptor_FC=Float64[])

pair_count = 0
for pair in common_pairs
    pair_count += 1
    println("$pair_count / $(length(common_pairs))")
    LR_pair_words = split(pair, "_")
    LR_pair_ligand = "Spatial:" * LR_pair_words[1]
    LR_pair_Receptor = "Spatial:" * LR_pair_words[2]
    
    ligand_exp_COI_mean = mean(spatial_object[COI_spots, LR_pair_ligand].X)
    ligand_exp_otherspots_mean = mean(spatial_object[rest_of_spots, LR_pair_ligand].X)
    ligand_FC = round(ligand_exp_COI_mean / ligand_exp_otherspots_mean, digits=4)
    
    Receptor_exp_COI_mean = mean(spatial_object[COI_spots, LR_pair_Receptor].X)
    Receptor_exp_otherspots_mean = mean(spatial_object[rest_of_spots, LR_pair_Receptor].X)
    Receptor_FC = round(Receptor_exp_COI_mean / Receptor_exp_otherspots_mean, digits=4)
    
    pair_p_gt = cooccur_res_df[(cooccur_res_df.pair12 .== pair) .| (cooccur_res_df.pair21 .== pair), :p_gt][1]
    ligand_index = findfirst(==(LR_pair_ligand), COI_cors_adata.var_names)
    receptor_index = findfirst(==(LR_pair_Receptor), COI_cors_adata.var_names)

    if pair_p_gt < 0.05 && COI_cors[ligand_index, receptor_index] > corr_threshold
        added_row = DataFrame(from=[LR_pair_ligand], to=[LR_pair_Receptor], correlation=[COI_cors[ligand_index, receptor_index]], ligand_FC=[ligand_FC], Receptor_FC=[Receptor_FC])
        append!(COI_enriched_LRs, added_row)
    end
end

1 / 2


2 / 2


In [25]:
COI_enriched_LRs

Row,from,to,correlation,ligand_FC,Receptor_FC
,String,String,Float64,Float64,Float64
1,Spatial:C3,Spatial:CXCR4,0.79157,1.3346,3.3869


In [26]:
# Sort the enriched LRs by correlation in decreasing order
sort!(COI_enriched_LRs, rev=true, [:correlation])

Row,from,to,correlation,ligand_FC,Receptor_FC
,String,String,Float64,Float64,Float64
1,Spatial:C3,Spatial:CXCR4,0.79157,1.3346,3.3869


In [27]:
# Add a 'pair' column to the enriched LRs DataFrame
COI_enriched_LRs[!, :pair] = string.(COI_enriched_LRs[!, :from], "_", COI_enriched_LRs[!, :to])
COI_enriched_LRs

Row,from,to,correlation,ligand_FC,Receptor_FC,pair
,String,String,Float64,Float64,Float64,String
1,Spatial:C3,Spatial:CXCR4,0.79157,1.3346,3.3869,Spatial:C3_Spatial:CXCR4


In [28]:
Output_dict = Dict("COI.enriched_LRs" => COI_enriched_LRs, "cooccurrence_table" => cooccur_COI_res)

Dict{String, Any} with 2 entries:
  "cooccurrence_table" => OrderedDict{Symbol, Any}(:call=>:(var"ISCHIA.cooccur"…
  "COI.enriched_LRs"   => 1×6 DataFrame…

In [ ]:







# Output_dict = Dict("COI.enriched_LRs" => COI_enriched_LRs, "cooccurrence_table" => cooccur_COI_res)

# return Output_dict

In [ ]:
# Define a function to calculate significant co-occurring Ligand-Receptor pairs
function enriched_LRs(spatial_object, COI, Condition, LR_list, LR_pairs, exp_th, corr_th)
    println("Preparing L-R presence/absence matrix")
    
    # Extract the expression matrix from spatial_object
    spatial_object_exp = spatial_object[:Spatial][:counts]
    spatial_object_exp_norm = spatial_object[:Spatial][:data]
    
    # Subset the expression matrix for the interested ligands and receptors
    spatial_obj_exp_LR_subset_raw = spatial_object_exp[LR_list, :]
    
    # Binarize the expression matrix based on the expression threshold
    spatial_obj_exp_LR_subset_raw_binary = spatial_obj_exp_LR_subset_raw .> exp_th
    spatial_obj_exp_LR_subset_raw_binary = spatial_obj_exp_LR_subset_raw_binary[:, sum(spatial_obj_exp_LR_subset_raw_binary, dims=1) .> 0]
    spatial_obj_exp_LR_subset_raw_binary = spatial_obj_exp_LR_subset_raw_binary[:, sum(spatial_obj_exp_LR_subset_raw_binary, dims=2) .> 0]
    LR_presence_absence_mat = spatial_obj_exp_LR_subset_raw_binary
    
    # Filter spots based on COI and Condition
    COI_spots = names(spatial_object[:CompositionCluster_CC][in(COI .& (spatial_object[:orig_ident] .== Condition))])
    rest_of_spots = setdiff(1:size(spatial_object, 1), COI_spots)
    
    println("Calculating L-R pairs correlation")
    COI_cors = cor(transpose(spatial_object_exp[LR_list, COI_spots]))
    COI_cors[isnan.(COI_cors)] .= 0.0
    
    println("Preparing for cooccurrence")
    common_spots = intersect(LR_list, COI_spots)
    coocur_COI_exp = DataFrame(transpose(Matrix(LR_presence_absence_mat[:, common_spots])))
    
    println("Cooccurrence calculation starts...")
    cooccur_COI_res = ISCHIA.cooccur(mat=coocur_COI_exp, type="spp_site", thresh=true, spp_names=true)
    println("Cooccurrence calculation ended")
    
    println("Summary of cooccurrence results:")
    println(summary(cooccur_COI_res))
    println("Probability table of cooccurrence results:")
    println(prob_table(cooccur_COI_res))
    
    # Add a 'pair' column to the result DataFrame
    cooccur_COI_res[:pair] = string.(cooccur_COI_res[:sp1_name], "_", cooccur_COI_res[:sp2_name])
    rownames!(cooccur_COI_res, cooccur_COI_res[:pair])
    
    common_pairs = intersect(LR_pairs, rownames(cooccur_COI_res))
    
    COI_enriched_LRs = DataFrame(from=String[], to=String[], correlation=Float64[], ligand_FC=Float64[], Receptor_FC=Float64[])
    
    pair_count = 0
    for pair in common_pairs
        pair_count += 1
        println("$pair_count / $(length(common_pairs))")
        LR_pair_words = split(pair, "_")
        LR_pair_ligand = LR_pair_words[1]
        LR_pair_Receptor = LR_pair_words[2]
        
        ligand_exp_COI_mean = mean(spatial_object_exp_norm[LR_pair_ligand, COI_spots])
        ligand_exp_otherspots_mean = mean(spatial_object_exp_norm[LR_pair_ligand, rest_of_spots])
        ligand_FC = round(ligand_exp_COI_mean / ligand_exp_otherspots_mean, digits=4)
        
        Receptor_exp_COI_mean = mean(spatial_object_exp_norm[LR_pair_Receptor, COI_spots])
        Receptor_exp_otherspots_mean = mean(spatial_object_exp_norm[LR_pair_Receptor, rest_of_spots])
        Receptor_FC = round(Receptor_exp_COI_mean / Receptor_exp_otherspots_mean, digits=4)
        
        if cooccur_COI_res[pair, "p_gt"] < 0.05 && COI_cors[LR_pair_ligand, LR_pair_Receptor] > corr_th
            added_row = DataFrame(from=[LR_pair_ligand], to=[LR_pair_Receptor], correlation=[COI_cors[LR_pair_ligand, LR_pair_Receptor]], ligand_FC=[ligand_FC], Receptor_FC=[Receptor_FC])
            append!(COI_enriched_LRs, added_row)
        end
    end
    
    # Sort the enriched LRs by correlation in decreasing order
    sort!(COI_enriched_LRs, rev=true, by=:correlation)
    
    # Add a 'pair' column to the enriched LRs DataFrame
    COI_enriched_LRs[:pair] = string.(COI_enriched_LRs[:from], "_", COI_enriched_LRs[:to])
    
    Output_dict = Dict("COI.enriched_LRs" => COI_enriched_LRs, "cooccurrence_table" => cooccur_COI_res)
    
    return Output_dict
end